![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fdata-viz-of-the-week&branch=main&subPath=water-quality/water-quality.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Callysto’s Weekly Data Visualization

## National Long-term Water Quality Statistics

### Recommended Grade levels: 9-12
<br>

### Instructions
#### “Run” the cells to see the graphs
Click “Cell” and select “Run All”.<br> This will import the data and run all the code, so you can see this week's data visualization. Scroll to the top after you’ve run the cells.<br> 

![instructions](https://github.com/callysto/data-viz-of-the-week/blob/main/images/instructions.png?raw=true)

**You don’t need to do any coding to view the visualizations**.
The plots generated in this notebook are interactive. You can hover over and click on elements to see more information. 

Email contact@callysto.ca if you experience issues.

### About this Notebook

Callysto's Weekly Data Visualization is a learning resource that aims to develop data literacy skills. We provide Grades 5-12 teachers and students with a data visualization, like a graph, to interpret. This companion resource walks learners through how the data visualization is created and interpreted by a data scientist. 

The steps of the data analysis process are listed below and applied to each weekly topic.

1. Question - What are we trying to answer? 
2. Gather - Find the data source(s) you will need. 
3. Organize - Arrange the data, so that you can easily explore it. 
4. Explore - Examine the data to look for evidence to answer the question. This includes creating visualizations. 
5. Interpret - Describe what's happening in the data visualization. 
6. Communicate - Explain how the evidence answers the question. 

# Question

Water is doubtlessly an essential natural resource for human survival. On that account, [Dietitians of Canada](https://www.dietitians.ca) recommends Canadians to consume on average 2.2 litres of water daily. Based on a recent report by [Statistics Canada](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3810004001), total water intake from all aquatic sources in 2017 was 3,767 million cubic metres. Hence, it is clear that any changes to water quality will significantly impact human health as well as economic and environmental conditions in Canada. 

Now that we know how crucial water is in our daily lives, we may wish to look deeper into the water quality in Canada. The water quality in Canada earns an "A" grade and ranks in 4th place compared to the 17 peer OECD countries. But how can we be sure of that?

Statistics Canada has consistantly been monitoring various parameters that could influence Canada's aquatic ecosystems. How can we utilize this observational data to investigate the water quality in Canada?

<p align="center">
  <img src="https://cdn.pixabay.com/photo/2017/05/27/06/17/water-2347799_960_720.jpg" width="600" />
</p>

### Goal
Our goal is to look at different chemical or physical parameters that reveal to us the quality of water reserves across Canada. We will investigate the spread of harmful parameters in water reserves across all Canadian provinces. Do Canadian water reserves demonstrate a consistant decrease in harmful parameters? If they don't, what interesting trends can be seen in our visualizations?

# Gather

### Code:
The code below will import the Python programming libraries we need to gather and organize the data to answer our question.

In [ ]:
# import libraries
%pip install -r requirements.txt
import pyodide_http
pyodide_http.patch_all()
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import IPython.display
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

### Data:

In this notebook, we will use long-term monitoring data from [Statistics Canada](https://data.ec.gc.ca/data/substances/monitor/national-long-term-water-quality-monitoring-data/). To measure the long-term water quality, Statistics Canada collected data on **145 distinct parameters** from **21 observation sites**.

Out of the 145 distinct water quality parameters, we will investigate **five parameters** in depth. Our parameters of interest are pH, dissolved oxygen, water temperature, dissolved sulphate and dissolved phosphorus. To understand each parameter more in-depth, refer to the table below. 

| **Parameter** | **Description** |
| :- |:------------- | 
| pH | A measure of the hydrogen ion activity in a solution. It indicates how acidic or basic water is. |
|  Water Temperature | Different kinds of aquatic organisms choose to live in varying water temperature. A change in water temperature will lead to changes in types of aquatic life present in water and triggers consecutive chain reactions. |
| Dissolved Oxygen | All aquatic organisms need dissolved oxygen for respiration. Low dissolved oxygen levels can  lead to increased mortality of fish eggs and devastates aquatic ecosystem that are present before. |
| Dissolved Sulphate | Sulphate is the oxidized form of sulphur, which is essential for many biological processes in plants and animals. However, high concentrations of sulphate are detrimental for the survival of aquatic organisms.  |
| Dissolved Phosphorus | Plants and animals need phosphorus to grow. However, high concentrations of phosphorus will enhance vegetation and algae growth, thus breaking the aquatic ecosystem. |

Description from the table was retrieved from [British Columbia government website](https://www2.gov.bc.ca/gov/content/environment/air-land-water/water/water-quality/water-quality-monitoring/canada-bc-water-quality-monitoring-program/water-quality-parameters). 

### Import the data

Since our entire dataset will be quite large, let's first look into data from a single location.  

In [ ]:
# import data
df = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/water-quality/data/okanagan.csv", encoding="latin-1")
print(df.shape)
df.head()

### Comment on the data
Notice that the data has **92418 rows** and **11 columns**. If we compile data from all 21 locations, the resulting dataframe will be massive! Therefore, before proceeding further, we will select specific columns and rows that are of our interest. 

# Organize
Now, we wish to gather data from all locations. Remember that we are only using five parameters, which we need to filter out in the process of data cleaning. Run the following cells to display long-term water quality monitoring data across all provinces in Canada. 

This data may take some time to load, so please wait paitiently.

In [ ]:
# Make a list of all locations
df = pd.DataFrame(columns = ["SITE_NO", "LOCATION", "YEAR", "VARIABLE", "VALUE_VALEUR", "UNIT_UNITE"])
observation_locations = ["arctic", "assiniboine", "churchill", "columbia", "fraser",
                         "keewatin", "mackenzie", "missouri", "south_saskatchewan",
                         "north_saskatchewan", "okanagan", "pacific_coastal",
                         "athabasca", "lower_saskatchewan", "winnipeg", "yukon",
                         "gaspe", "maritime", "newfoundland", "st_john", "st_lawrence"]

# Create dictionaries to replace certain entries
prov = {"Newfoundland and Labrador": "NL", "Prince Edward Island":"PE",
       "Nova Scotia":"NS", "New Brunswick":"NB", "Quebec":"QC", "Ontario":"ON",
       "Manitoba":"MB", "Alberta":"AB", "Saskatchewan":"SK", "British Columbia":"BC",
        "Northwest Territories":"NT", "Nunavut":"NU", "Yukon":"YT"}
prov_reverse = {y: x for x, y in prov.items()}

units = {"PH UNITS":"PH",
       "MG/L":"mg/L",
       "DEG C":"°C"}

parameter_rename = {"Phosphorus Total Dissolved":"Dissolved Phosphorus",
                   "Temperature Water":"Water Temperature",
                   "PH":"pH",
                   "Oxygen Dissolved":"Dissolved Oxygen",
                   "Sulphate Dissolved":"Dissolved Sulphate"}

# Gather all observed data
def data_wrangling(lists):
    for alist in lists:
        mydf = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/water-quality/data/" + alist + ".csv", encoding='latin-1')
        mydf["LOCATION"] = str(alist)
        mydf["DATE_TIME_HEURE"] = pd.to_datetime(mydf["DATE_TIME_HEURE"])
        mydf["YEAR"] = pd.DatetimeIndex(mydf["DATE_TIME_HEURE"]).year
        mydf = mydf[["SITE_NO", "YEAR", "LOCATION", "VARIABLE", "VALUE_VALEUR", "UNIT_UNITE", "VMV_CODE"]]
        mydf1 = mydf.groupby(["YEAR", "LOCATION", "SITE_NO", "VARIABLE", "UNIT_UNITE", "VMV_CODE"])["VALUE_VALEUR"].mean()
        mydf1 = mydf1.to_frame().reset_index()
        global df
        df = pd.concat([df, mydf1])
    return df

# Add longitude and latitude for each location
sites = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/water-quality/data/site_lists.csv", encoding='latin-1')
pre_df = data_wrangling(observation_locations)
pre_df = pre_df.drop(pre_df[pre_df["VMV_CODE"] == 2061.0].index)
df = pd.merge(pre_df, sites, on="SITE_NO", how="inner")

# Select parameters
parameters = ["PH", "OXYGEN DISSOLVED", "TEMPERATURE WATER", "PHOSPHORUS TOTAL DISSOLVED", "SULPHATE DISSOLVED"]
df = df[df.VARIABLE.isin(parameters)]

# Eliminate out of range values
PH_out_of_range = df[(df['VARIABLE']=='PH') & (~df["VALUE_VALEUR"].between(1,14))].index.values.tolist()
temp_water_out_of_range = df[(df['VARIABLE']=='TEMPERATURE WATER') & (df["VALUE_VALEUR"]<0)].index.values.tolist()
oxygen_out_of_range = df[(df['VARIABLE']=='OXYGEN DISSOLVED') & (df["VALUE_VALEUR"]>20)].index.values.tolist()
all_out_of_range = PH_out_of_range + temp_water_out_of_range + oxygen_out_of_range
df.drop(all_out_of_range, inplace=True)

# Rename columns and further data cleaning 
df = df.rename(columns={"VALUE_VALEUR":"VALUE", "PROV_TERR":"PROVINCE", "UNIT_UNITE":"UNIT"})
df["PROVINCE"] = df["PROVINCE"].replace(prov_reverse)
df["UNIT"] = df["UNIT"].replace(units)
df = df[["PROVINCE", "LOCATION", "YEAR", "SITE_NAME", "SITE_TYPE", "LONGITUDE", "LATITUDE", "VARIABLE", "VALUE", "UNIT"]].reset_index(drop=True)
df["VARIABLE"] = df["VARIABLE"].apply(lambda x: x.title() if (x!="PH") else x)
df["VARIABLE"] = df["VARIABLE"].replace(parameter_rename, regex=True)
df = df.loc[df["YEAR"] < 2020]

# Assign specific colors for each province 
colors=['#1f77b4',  # muted blue
        '#ff7f0e',  # safety orange
        '#2ca02c',  # cooked asparagus green
        '#d62728',  # brick red
        '#9467bd',  # muted purple
        '#8c564b',  # chestnut brown
        '#e377c2',  # raspberry yogurt pink
        '#17becf',  # blue-teal
        '#210240',  # dark purple
        '#21DC49',  # bright green
        '#3F5063',  # dark navy
        '#6C7075',  # dark grey
        '#F4BC1A']  # mustard
    
color_dict = dict(zip(df["PROVINCE"].unique(), colors))
df["COLOR"] = df["PROVINCE"].map(color_dict)

df.head()

### Comment on the data
The filtered dataframe consists of **14975 rows** and **10 columns**. This number is significantly smaller than what would have occurred if all 145 parameter data from 21 observation locations had been compiled. 

We can also look into the **provincial average** of the parameter concentrations (values). The dataframe below will display a summarized form of the previous dataframe, with the provincial average values for each year, for the five selected parameters. 

In [ ]:
# Provincial summary of yearly average parameter concentrations
df_summary = df.groupby(["PROVINCE", "VARIABLE", "YEAR", "UNIT"])["VALUE"].mean().to_frame()
df_summary = df_summary.reset_index()
print(df_summary.shape)
df_summary.head()

The summarized dataframe consists of **1024 rows** and **5 columns**. We will use the summarized dataframe to create a line graph and the original dataframe to create a series of data-plotted maps. From now, we will refer to the summarized dataframe as `df_summary`, and the original dataframe as `df`.

# Explore

### Line Graph 
We will start by creating a **line graph**. To make this graph, we will use `df_summary`, which contains provincial average data of parameter concentrations from 2000 to 2019. Notice that some of the data is missing; when you see discontinued lines in the plot, it means that the particular data is missing.  

In [ ]:
line_fig = go.Figure()
variables = df_summary["VARIABLE"].unique()
firstvis = lambda x, y: True if ((x=="Alberta") & (y=="Dissolved Oxygen")) else "legendonly" if (y=="Dissolved Oxygen") else False

markers = ["circle","square","diamond","cross","x","triangle-up",
           "triangle-down","triangle-left","triangle-right","star","hourglass",
           "hexagram", "bowtie"]

for variable in variables:
    col_num = 0
    df1 = df_summary[df_summary["VARIABLE"] == variable]
    all_provs = df1["PROVINCE"].unique()
    for ind, province in enumerate(all_provs):
        customdata = np.stack((df1[df1["PROVINCE"] == province]["PROVINCE"], 
                       df1[df1["PROVINCE"] == province]["VALUE"], 
                       df1[df1["PROVINCE"] == province]["UNIT"],
                       df1[df1["PROVINCE"] == province]["YEAR"]), axis=1)
        hovertemplate="<br>".join(["<b>Province:  %{customdata[0]}</b>", "Value: %{customdata[1]}" + " %{customdata[2]}<extra></extra>"])
        line_fig.add_trace(go.Scatter(x=df1[df1["PROVINCE"] == province]["YEAR"],
                                      y=df1[df1["PROVINCE"] == province]["VALUE"],                                      
                                      mode = "lines+markers", uid=variable+ "(" + province + ")",
                                      customdata = customdata,
                                      line=dict(color=colors[col_num]), 
                                      marker_symbol=markers[ind], marker_size=10,
                                      hovertemplate = hovertemplate,
                                      hoverinfo = "x",
                                      name= province, visible= firstvis(province, variable)))
        col_num += 1
    
    
mybuttons = []
prov_var_dict = [[] for _ in range(len(variables))]

for ind_variable, variable in enumerate(variables):
    df1 = df_summary[df_summary["VARIABLE"] == variable]
    all_provs = df1["PROVINCE"].unique()
    for province in all_provs:
        prov_var_dict[ind_variable].append(province)
        
currnum = 0
for ind, variable in enumerate(prov_var_dict):
    traces = [False] * len(line_fig.data)
    num_provs = len(variable)
    oldnum = currnum
    traces[0:oldnum] = [False for i in traces[0:oldnum]]
    currnum += num_provs
    traces[oldnum:currnum] = ["legendonly" for i in traces[oldnum:currnum]]
    traces[oldnum] = [True]
    mybuttons.append(dict(label=variables[ind],
                          method="update",
                          args=[{"visible":traces}]))

line_fig.update_layout(showlegend=True,
                      updatemenus=[dict(active=0,
                                        buttons=mybuttons,
                                        direction="down", pad={"r": 10, "t": 10},
                                        showactive=True, x=0.15, xanchor="left", y=1.15, yanchor="top")],
                      annotations=[dict(text="Parameter: ",x=0, y=1.09, yref="paper", xref="paper", align="left", showarrow=False)],
                      xaxis_title="Year", yaxis_title="Physical-Chemical Parameters Value",
                       xaxis_range=[1999, 2020],
                      title_text="Water Quality Across Provinces 2000-2019", title_x=0.5, width=800, height=500,
                      hovermode="x unified")
line_fig.show()

### Data Cleaning

Looking at the line graphs above, there a few different parameters where we can see some strange values. For example, in a few of the available categories from the dropdown list, the measurements for Alberta seem to have some anomalous (or unexpected) deviations from the general pattern. Alberta isn't the only region whose data contains these oddities, they're often a reality of collecting data from natural phenomena; some values will buck the normal trend.

Thankfully, because this is so common in data science (and science in general), tackling this issue is a routine part of the exploration process. Identifying abnormal data, or what we might call **outliers**, is a simple task, but choosing what to do with data identified as an outlier is much trickier. It's almost always bad practice to remove data without a good reason, and in science it could even discredit your research. To justify our decisions here, we've found data from the [Canadian Council of Ministers of the Environment](https://ccme.ca/en/resources) that list reference ranges for all the parameters, though there's no levels given for water temperature (links open to PDFs):
- [pH](https://ccme.ca/en/res/ph-marine-en-canadian-water-quality-guidelines-for-the-protection-of-aquatic-life.pdf)
- [Dissolved Phosphorus](https://ccme.ca/en/res/phosphorus-en-canadian-water-quality-guidelines-for-the-protection-of-aquatic-life.pdf)
- [Dissolved Sulphate](https://www2.gov.bc.ca/assets/gov/environment/air-land-water/water/waterquality/water-quality-guidelines/approved-wqgs/sulphate/bc_moe_wqg_sulphate.pdf)
- [Water Temperature](https://ccme.ca/en/res/temperature-marine-en-canadian-water-quality-guidelines-for-the-protection-of-aquatic-life.pdf)
- [Dissolved Oxygen](https://ccme.ca/en/res/dissolved-oxygen-freshwater-en-canadian-water-quality-guidelines-for-the-protection-of-aquatic-life.pdf)

Below we have box plots that show the range of all the values for each parameter and location, to get an idea of the distributions. If you're not familiar with [box plots](https://en.wikipedia.org/wiki/Box_plot), they're a common visualization used to show *where* in a range the values are most grouped, by allowing you to see useful measures from the data. Statistical information, such as the median, interquartile range (IQR), and outliers (determined by default as 1.5 x IQR above or below the edges of the box) are easy to find by hovering over the plots.

Overlaid in grey is the reference range for each parameter from the above sources. **CLICK on the left of the graph to unscroll the output for all the graphs.**

In [ ]:
fig2 = px.box(df[df['VARIABLE']=='pH'], x='VALUE', title='pH', color='PROVINCE', height=600)
fig2.add_shape(type='rect', opacity=0.2, fillcolor='grey',
              yref='paper',
              x0=7.0, x1=8.7,
              y0=0.0, y1=1.0)
fig2.show()
fig3 = px.box(df[df['VARIABLE']=='Dissolved Phosphorus'], x='VALUE', title='Dissolved Phosphorus', color='PROVINCE', height=600)
fig3.add_shape(type='rect', opacity=0.2, fillcolor='grey',
              yref='paper',
              x0=0.0, x1=0.1,
              y0=0.0, y1=1.0)
fig3.show()
fig4 = px.box(df[df['VARIABLE']=='Dissolved Sulphate'], x='VALUE', title='Dissolved Sulphate', color='PROVINCE', height=600)
fig4.add_shape(type='rect', opacity=0.2, fillcolor='grey',
              yref='paper',
              x0=0.0, x1=250,
              y0=0.0, y1=1.0)
fig4.show()
fig5 = px.box(df[df['VARIABLE']=='Water Temperature'], x='VALUE', title='Water Temperature', color='PROVINCE', height=600)

fig5.show()
fig6 = px.box(df[df['VARIABLE']=='Dissolved Oxygen'], x='VALUE', title='Dissolved Oxygen', color='PROVINCE', height=600)
fig6.add_shape(type='rect', opacity=0.2, fillcolor='grey',
              yref='paper',
              x0=5.5, x1=12.77,
              y0=0.0, y1=1.0)
fig6.show()

#### A note on cleaning
When doing data cleaning, there's no 'one size fits all' approach on how to handle outliers. Though there are many techniques and approaches that do exist, often the best method is determined by convention in each field. Even before that, it's usually good practice to discard data points that are physically impossible, such as measuring a person's height at 20m, or an air temperature of 350°C.

If we were professional scientists in the field of environmental science, a better approach would be to look at the current research in the field and see what other scientists have done to handle outliers. For our purposes here though, we can use the reference ranges given to us to determine which values to keep. This will impact some regions and parameters more than others, but it's a quick way to clean our data.

Below we'll remove outlier values from the original data, recombine it, and plot it again.

In [ ]:
# Remove rows with values out of range for each parameter
ph_drop = df.index[((df['VARIABLE']=='pH') & (df['VALUE'] > 8.7)) | 
                   ((df['VARIABLE']=='pH') & (df['VALUE'] < 7.0))].to_list()
phos_drop = df.index[(df['VARIABLE']=='Dissolved Phosphorus') & (df['VALUE'] > 0.1)].to_list()
sulph_drop = df.index[(df['VARIABLE']=='Dissolved Sulphate') & (df['VALUE'] > 250)].to_list()
oxy_drop = df.index[((df['VARIABLE']=='Dissolved Oxygen') & (df['VALUE'] < 5.5)) |
                    ((df['VARIABLE']=='Dissolved Oxygen') & (df['VALUE'] > 12.7))].to_list()
drops = ph_drop + phos_drop + sulph_drop + oxy_drop

df_cleaned = df.drop(drops, axis=0)

# Re-summarize data
df_cleaned_summary = df_cleaned.groupby(["PROVINCE", "VARIABLE", "YEAR", "UNIT"])["VALUE"].mean().to_frame()
df_cleaned_summary = df_cleaned_summary.reset_index()


# Plot it again
line_fig2 = go.Figure()
variables = df_cleaned_summary["VARIABLE"].unique()
firstvis = lambda x, y: True if ((x=="Alberta") & (y=="Dissolved Oxygen")) else "legendonly" if (y=="Dissolved Oxygen") else False

markers = ["circle","square","diamond","cross","x","triangle-up",
           "triangle-down","triangle-left","triangle-right","star","hourglass",
           "hexagram", "bowtie"]

for variable in variables:
    col_num = 0
    df1 = df_cleaned_summary[df_cleaned_summary["VARIABLE"] == variable]
    all_provs = df1["PROVINCE"].unique()
    for ind, province in enumerate(all_provs):
        customdata = np.stack((df1[df1["PROVINCE"] == province]["PROVINCE"], 
                       df1[df1["PROVINCE"] == province]["VALUE"], 
                       df1[df1["PROVINCE"] == province]["UNIT"],
                       df1[df1["PROVINCE"] == province]["YEAR"]), axis=1)
        hovertemplate="<br>".join(["<b>Province:  %{customdata[0]}</b>", "Value: %{customdata[1]}" + " %{customdata[2]}<extra></extra>"])
        line_fig2.add_trace(go.Scatter(x=df1[df1["PROVINCE"] == province]["YEAR"],
                                      y=df1[df1["PROVINCE"] == province]["VALUE"],                                      
                                      mode = "lines+markers", uid=variable+ "(" + province + ")",
                                      customdata = customdata,
                                      line=dict(color=colors[col_num]), 
                                      marker_symbol=markers[ind], marker_size=10,
                                      hovertemplate = hovertemplate,
                                      hoverinfo = "x",
                                      name= province, visible= firstvis(province, variable)))
        col_num += 1
    
    
mybuttons = []
prov_var_dict = [[] for _ in range(len(variables))]

for ind_variable, variable in enumerate(variables):
    df1 = df_cleaned_summary[df_cleaned_summary["VARIABLE"] == variable]
    all_provs = df1["PROVINCE"].unique()
    for province in all_provs:
        prov_var_dict[ind_variable].append(province)
        
currnum = 0
for ind, variable in enumerate(prov_var_dict):
    traces = [False] * len(line_fig2.data)
    num_provs = len(variable)
    oldnum = currnum
    traces[0:oldnum] = [False for i in traces[0:oldnum]]
    currnum += num_provs
    traces[oldnum:currnum] = ["legendonly" for i in traces[oldnum:currnum]]
    traces[oldnum] = [True]
    mybuttons.append(dict(label=variables[ind],
                          method="update",
                          args=[{"visible":traces}]))

line_fig2.update_layout(showlegend=True,
                      updatemenus=[dict(active=0,
                                        buttons=mybuttons,
                                        direction="down", pad={"r": 10, "t": 10},
                                        showactive=True, x=0.15, xanchor="left", y=1.15, yanchor="top")],
                      annotations=[dict(text="Parameter: ",x=0, y=1.09, yref="paper", xref="paper", align="left", showarrow=False)],
                      xaxis_title="Year", yaxis_title="Physical-Chemical Parameters Value",
                       xaxis_range=[1999, 2020],
                      title_text="Water Quality Across Provinces 2000-2019", title_x=0.5, width=800, height=500,
                      hovermode="x unified")
line_fig2.show()

The steps we took in cleaning didn't remove **all** the variation in the data, and we wouldn't want it to, but it does help to know that the displayed data only contains values within the range of normal.

### Data-plotted map
We will now learn to create a **map** with some data plotting. In this graph, the size of the bubbles will represent the parameter concentration or values. For the ease of comparison, we categorized all parameter values into 3 groups: **< First quartile, > Second quartile**, and **> Third quartile**. We will start by creating some functions that allow us to categorize the parameter values. Image [credit and source](https://online.stat.psu.edu/stat500/sites/stat500/files/inline-images/500%20l1%2025th%20and%2075th%20percentile.png).  

<p align="center">
  <img src="https://online.stat.psu.edu/stat500/sites/stat500/files/inline-images/500%20l1%2025th%20and%2075th%20percentile.png" width="600" />
</p>

**Percentile** is a statistical term that divides the entire spectrum of values (population) according to the distribution of values of a particular variable. Each water quality monitoring parameter will have a different set of percentile values. 

- Data points that are in the `< First Quartile` category are the **lower 25%** (percentile) of the value spectrum.
- Data points that are in the `> Second Quartile` category are the **25%~75%** (percentile) of the value spectrum.
- Data points that are in the `> Third Quartile` category are the **upper 25%** (percentile) of the value spectrum. 

In [ ]:
# Define a function to assign category to parameter values
def create_scale(all_values, value):
    small = np.percentile(value, 25)
    large = np.percentile(value, 75)
    if (all_values <= small): return "< First Percentile"
    elif (all_values > small) & (all_values <= large): return "> First Percentile"
    elif (all_values >= large): return "> Third Percentile"

# Apply the scale to associated values 
def apply_scale(all_values):
    num_small = 5
    num_medium = 10
    num_large = 20
    if (all_values == "< First Percentile"): return num_small
    elif (all_values == "> First Percentile"): return num_medium
    elif (all_values == "> Third Percentile"): return num_large

Now we create a **map** to view the parameter concentrations or values in specific locations based on the longitude and latitude. We will define a **function**, `create_map_plot`, which can be used on any of the aforementioned parameters.

In [ ]:
def create_map_plot(any_df, variable):
    df1 = any_df[any_df["VARIABLE"] == variable].copy().sort_values('YEAR')

    quantile_value = df1.VALUE.astype(float)
    df1["Category"] = df1["VALUE"].apply(lambda x: create_scale(x, quantile_value))
    df1["Scaled"] = df1["Category"].apply(lambda x: apply_scale(x))
    
    customdata = [df1["PROVINCE"], df1["LOCATION"], df1["VARIABLE"], df1["VALUE"], df1["UNIT"],
                  df1["LONGITUDE"], df1["LATITUDE"], df1["Category"]]
    
    hovertemplate="<br>".join([
        "<b>Province:  %{customdata[0]}</b>",
        "Longitude: %{customdata[5]}",
        "Latitude: %{customdata[6]}",
        "Location: %{customdata[1]}",
        "Value: %{customdata[3]}",
        "Unit: %{customdata[4]}",
        "Category: %{customdata[7]}<extra></extra>"])

    fig = px.scatter_mapbox(df1,
                            lat="LATITUDE",
                            lon="LONGITUDE",
                            color="PROVINCE",
                            color_discrete_sequence = colors,
                            animation_frame="YEAR",
                            size=df1["Scaled"],
                            hover_name="VALUE",
                            custom_data = customdata,
                            opacity=0.6).update_layout(mapbox={"style": "carto-positron", "zoom": 2, 
                                                               "center":{'lon':-98, 'lat': 61}}, 
                                                       margin={"t":0,"b":0,"l":0,"r":0})
    
    fig.update_traces(hovertemplate=hovertemplate)
    
    for frame in fig.frames:
        for data in frame.data:
            data.hovertemplate = hovertemplate
    
    return fig

With the defined function, `create_map_plot`, we can now apply it to the `pH` parameter. The size of the bubbles are indicative of their basicity; the larger the bubbles are, the more basic the water is. 

In [ ]:
pH_mapped = create_map_plot(df, "pH")
pH_mapped

We can now also plot the parameter `Dissolved Oxygen` using the function above. The size of the bubbles are representative of the total dissolved oxygen concentration in the water. The larger the bubbles are, the higher oxygen concentration there is. 

This function will work on any of the five selected parameters. 

In [ ]:
oxygen_dissolved_mapped = create_map_plot(df, "Dissolved Oxygen")
oxygen_dissolved_mapped

As a last step, we will plot the parameter `Water Temperature`. Do you see any consistencies from the three maps created?

In [ ]:
temp_water_mapped = create_map_plot(df, "Water Temperature")
temp_water_mapped

### Data-plotted map with *ipywidgets*
With the help of **ipywidget**, we can also create a dropdown menu that allows us to switch back and forth between parameters on a map diagram. Unlike the previous diagrams, where we used **Plotly Express**, we will be using **Plotly Graph Objects**.

Let's start by assigning each value into the correct percentile category. 

In [ ]:
# Assign percentile category on all data
def add_category(df):
    df["Category"] = "NA"
    df["Scale"] = np.NaN
    for variable in df["VARIABLE"].unique():
        mydf = df[df["VARIABLE"] == variable]
        quantile_variable = mydf.VALUE.astype(float)
        for index, row in df.iterrows():
            if row.VARIABLE == variable:
                myval = row.VALUE
                category = create_scale(myval, quantile_variable)
                df.at[index, "Category"] = category
                scaled_value = apply_scale(category)
                df.at[index, "Scale"] = scaled_value
    return df

In [ ]:
# Create a scatterplot 
def create_map_with_widget(df):
    add_category(df)
    years = df["YEAR"].unique().tolist()

    hovertemplated="<br>".join([
        "<b>Province:  %{customdata[0]}</b>",
        "Longitude: %{customdata[2]}",
        "Latitude: %{customdata[1]}",
        "Location: %{customdata[3]}",
        "Variable: %{customdata[4]}",
        "Value: %{customdata[5]}",
        "Unit: %{customdata[6]}",
        "Category: %{customdata[7]}"])

    frames = [{"name":"frame_{}".format(year),
               "data":[{"type":"scattermapbox",
                        "lat":df.loc[df["YEAR"] == year]["LATITUDE"],
                        "lon":df.loc[df["YEAR"] == year]["LONGITUDE"],
                        "marker":go.scattermapbox.Marker(
                            size=df.loc[df["YEAR"] == year]["Scale"],
                            color=df.loc[df["YEAR"] == year]["COLOR"]),
                        "customdata":np.stack((df.loc[df["YEAR"] == year]["PROVINCE"],
                                   df.loc[df["YEAR"] == year]["LATITUDE"],
                                   df.loc[df["YEAR"] == year]["LONGITUDE"],
                                   df.loc[df["YEAR"] == year]["LOCATION"],
                                   df.loc[df["YEAR"] == year]["VARIABLE"],
                                   df.loc[df["YEAR"] == year]["VALUE"],
                                   df.loc[df["YEAR"] == year]["UNIT"],
                                   df.loc[df["YEAR"] == year]["Category"]), axis=1),
                        "hovertemplate": hovertemplated + "<extra></extra>"}]} for year in years]

    sliders = [{"transition":{"duration":0}, "x":0.08, "len":0.9,
                "currentvalue":{"font":{"size":12},"prefix":"Year: ", "visible":True, "xanchor":"left"},
                "steps":[{"label":year, "method":"animate",
                          "args":[["frame_{}".format(year)],
                                  {'mode':'immediate', 'frame':{'duration':100, 'redraw': True}, 
                                   'transition':{'duration':50}}]} for year in years]}]

    play_button = [{'type':'buttons','showactive':True,'x':0.045, 'y':-0.1,
                    'buttons':[{'label':'►','method':'animate',
                                'args':[None,{'frame':{'duration':100, 'redraw':True},
                                              'transition':{'duration':65},
                                              'fromcurrent':True,'mode':'immediate'}]}]}]

    layout = go.Layout(sliders=sliders, updatemenus=play_button,
                       mapbox = {'style': "carto-positron", 'center': {'lon':-98, 'lat': 58},'zoom': 2.5})

    myfig = go.Figure(data=frames[0]["data"], layout=layout, frames=frames)
    myfig.update_layout(width=800, height=500, margin=dict(l=20, r=20, t=0, b=20))
    myfig.show()

# Link the ipywidget with parameters and created scatterplot
variables = widgets.Dropdown(options=sorted(df.VARIABLE.unique()), 
                             value=df["VARIABLE"].unique().tolist()[0], description="Variable: ")

def created_map_with_widget(change):
    IPython.display.clear_output(wait=True)
    df1 = df.loc[df["VARIABLE"] == variables.value].copy().sort_values('YEAR')
    display(variables)
    create_map_with_widget(df1)


variables.observe(created_map_with_widget, names="value")
created_map_with_widget("")

# Interpret
What conclusions can we draw from the visualizations above? Did we see any noticeable changes over the years? Which province has the ideal water conditions? 

#### Line Graph
- Overall, parameter concentrations and values seem to stay the same. Although they fluctuate yearly, there isn't a huge change in the trend. One interesting point to note is that **Quebec** showed a distinguishably high concentration of dissolved phosphorus compared to other provinces. 

#### Data-Plotted Map Graph
- Overall, we notice the same trend as seen in the line graph, but this type of graph allows you to look into specific locations by latitude and longitude. We also notice that far more observations are done on **British Columbia** compared to other provinces. Do you think this could influence the outcome of the visualization?

As seen in the graphs, each graph has pros and cons. What do you think is the best type of graph to show Canadian water quality? If you could change the graph, how would you display the data?

# Communicate
Below are some writing prompts to help you reflect on the new information that is presented from the data. When we look at the evidence, think about what you perceive about the information. Is this perception based on what the evidence shows? If others were to view it, what perceptions might they have?

- I used to think __ but now I think ________.
- I wish I knew more about __.
- This visualization reminds me of __.
- I really like __.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)